In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

# 首先生成各店铺每月的月销售量

In [ ]:
t_order=pd.read_csv('t_order_everyday.csv')
t_order['ord_dt'] = pd.to_datetime(t_order['ord_dt'])

In [ ]:
date_list = ['2016/6/30','2016/7/31','2016/8/31','2016/9/30','2016/10/31','2016/11/30','2016/12/31','2017/1/31','2017/2/28','2017/3/31','2017/4/30']
date_list = pd.to_datetime(date_list)

In [ ]:
feature = pd.DataFrame()
for i in range(1,len(date_list)):
    time_interval = t_order[(date_list[i-1] < t_order['ord_dt']) & (t_order['ord_dt'] <= date_list[i])]
    #sale_amt
    sale_amt = time_interval.groupby('shop_id', as_index=False).agg({'sale_amt':['sum']})
    sale_amt.columns = ['shop_id','label']
    sale_amt['dt'] = date_list[i]
    feature = pd.concat([feature, sale_amt])
    print(date_list[i])

In [ ]:
feature.to_csv('month_label.csv',index=False)

# 提取各店铺产品的种类信息

In [ ]:
t_product=pd.read_csv('t_product.csv')

In [ ]:
cate_product=t_product[['shop_id','cate']]
cate_product=cate_product.drop_duplicates(['shop_id','cate'])

In [ ]:
cate_product=cate_product.groupby('shop_id',as_index=False).agg({'cate':['count']})
cate_product.columns = ['shop_id','cate_num']

In [ ]:
pid_product=t_product[['shop_id','pid']]
pid_product=pid_product.drop_duplicates(['shop_id','pid'])
pid_product=pid_product.groupby('shop_id',as_index=False).agg({'pid':['count']})
pid_product.columns = ['shop_id','pid_num']

# 开始生成前一个月产品的基本信息

In [ ]:
feature1 = pd.DataFrame()
for i in range(1,len(date_list)):
    time_interval = t_order[(date_list[i-1] < t_order['ord_dt']) & (t_order['ord_dt'] <= date_list[i])]
    #sale_amt
    sale_amt = time_interval.groupby('shop_id', as_index=False).agg({'sale_amt':['sum']})
    sale_amt.columns = ['shop_id','sale_month']
    #offer_amt
    offer_amt = time_interval.groupby('shop_id', as_index=False).agg({'offer_amt':['sum']})
    offer_amt.columns = ['shop_id', 'offer_amt_month']
    all_feature = pd.merge(sale_amt,offer_amt,on='shop_id',how='left')
    #offer_cnt
    offer_cnt = time_interval.groupby('shop_id', as_index=False).agg({'offer_cnt':['sum']})
    offer_cnt.columns = ['shop_id', 'offer_cnt_month']
    all_feature = pd.merge(all_feature, offer_cnt,on='shop_id',how='left')
    #rtn_cnt
    rtn_cnt = time_interval.groupby('shop_id', as_index=False).agg({'rtn_cnt':['sum']})
    rtn_cnt.columns = ['shop_id', 'rtn_cnt_month']
    all_feature = pd.merge(all_feature, rtn_cnt, on='shop_id',how='left')
    #rtn_cmt
    rtn_amt = time_interval.groupby('shop_id', as_index=False).agg({'rtn_amt':['sum']})
    rtn_amt.columns = ['shop_id', 'rtn_amt_month']
    all_feature = pd.merge(all_feature, rtn_amt, on='shop_id',how='left')
    #ord_cnt
    ord_cnt = time_interval.groupby('shop_id', as_index=False).agg({'ord_cnt':['sum']})
    ord_cnt.columns = ['shop_id', 'ord_cnt_month']
    all_feature = pd.merge(all_feature, ord_cnt, on='shop_id',how='left')
    #usr_cnt
    user_cnt = time_interval.groupby('shop_id', as_index=False).agg({'user_cnt':['sum']})
    user_cnt.columns = ['shop_id', 'user_cnt_month']
    all_feature = pd.merge(all_feature, user_cnt, on='shop_id',how='left')
    all_feature['dt'] = date_list[i]
    feature1 = pd.concat([feature1, all_feature])
    print(date_list[i])

In [ ]:
feature1['priduct_price']=feature1.sale_month/feature1.ord_cnt_month

In [ ]:
#各店铺平均销售额特征
sale_mean=feature1[['shop_id','sale_month']]
sale_mean=sale_mean.groupby('shop_id',as_index=False).agg({'sale_month':['mean','max','std']})
sale_mean.columns = ['shop_id','sale_mean','sale_max','sale_std']

In [ ]:
#各店铺平均订单量特征
ord_mean=feature1[['shop_id','ord_cnt_month']]
ord_mean=ord_mean.groupby('shop_id',as_index=False).agg({'ord_cnt_month':['mean','max','std']})
ord_mean.columns = ['shop_id','ord_mean','ord_max','std']

In [ ]:
#各店铺平均价格特征
price_mean=feature1[['shop_id','priduct_price']]
price_mean=price_mean.groupby('shop_id',as_index=False).agg({'priduct_price':['mean','max','std']})
price_mean.columns = ['shop_id','price_mean','price_max','price_std']

In [ ]:
#各店铺平均优惠特征
offer_mean=feature1[['shop_id','offer_cnt_month']]
offer_mean=offer_mean.groupby('shop_id',as_index=False).agg({'offer_cnt_month':['mean','max']})
offer_mean.columns = ['shop_id','offer_mean','offer_max']

In [ ]:
comment= pd.read_csv('t_comment.csv')

In [ ]:
#各店铺的评价特征信息
bad_num = comment.groupby('shop_id', as_index=False).agg({'bad_num':['sum']})
bad_num.columns = ['shop_id', 'bad_num_sum']
#cmmt_num
cmmt_num = comment.groupby('shop_id', as_index=False).agg({'cmmt_num': ['sum']})
cmmt_num.columns = ['shop_id', 'cmmt_num_sum']
all_feature = pd.merge(bad_num, cmmt_num, on='shop_id')
#dis_num
dis_num= comment.groupby('shop_id', as_index=False).agg({'dis_num': ['sum']})
dis_num.columns = ['shop_id', 'dis_num_sum']
all_feature = pd.merge(all_feature, dis_num, on='shop_id')
#good_num
good_num = comment.groupby('shop_id', as_index=False).agg({'good_num': ['sum']})
good_num.columns = ['shop_id', 'good_num_sum']
all_feature = pd.merge(all_feature, good_num, on='shop_id')
#mid_num
mid_num = comment.groupby('shop_id', as_index=False).agg({'mid_num': ['sum']})
mid_num.columns = ['shop_id', 'mid_num_sum']
all_feature = pd.merge(all_feature, mid_num, on='shop_id')

In [ ]:
all_feature['comment_num']=all_feature.bad_num_sum+all_feature.good_num_sum+all_feature.mid_num_sum
all_feature['good_rate']=all_feature.good_num_sum/all_feature.comment_num

In [ ]:
#各店铺广告特征提取
t_ads=pd.read_csv('t_ads.csv')
t_ads['create_dt'] = pd.to_datetime(t_ads['create_dt'])

In [ ]:
date_list = ['2016/6/30','2016/7/31','2016/8/31','2016/9/30','2016/10/31','2016/11/30','2016/12/31','2017/1/31','2017/2/28','2017/3/31','2017/4/30']
date_list = pd.to_datetime(date_list)

In [ ]:
feature2 = pd.DataFrame()
for i in range(1,len(date_list)):
    time_interval =t_ads[(date_list[i-1] < t_ads['create_dt']) & (t_ads['create_dt'] <= date_list[i])]
    #bad_amt
    consume_feature = time_interval.groupby('shop_id', as_index=False).agg({'consume':['sum']})
    consume_feature.columns = ['shop_id', 'con_sum']
    consume_feature['dt'] = date_list[i]
    feature2 = pd.concat([feature2, consume_feature])
    print(date_list[i])

In [ ]:
#各店铺投放广告的次数
ads_times=feature2[['shop_id','con_sum']]
ads_times=ads_times.groupby('shop_id',as_index=False).agg({'con_sum':['count']})
ads_times.columns = ['shop_id','ads_times']

In [ ]:
#各店铺投放广告金额
ads_mean=feature2[['shop_id','con_sum']]
ads_mean=ads_mean.groupby('shop_id',as_index=False).agg({'con_sum':['mean']})
ads_mean.columns = ['shop_id','ads_mean']

In [ ]:
shop_feature=pd.merge(sale_mean,ord_mean,on='shop_id',how='left')
shop_feature=pd.merge(shop_feature,price_mean,on='shop_id',how='left')
shop_feature=pd.merge(shop_feature,offer_mean,on='shop_id',how='left')
shop_feature=pd.merge(shop_feature,all_feature[['shop_id','comment_num','dis_num_sum','good_rate']],on='shop_id',how='left')
shop_feature=pd.merge(shop_feature,ads_times,on='shop_id',how='left')
shop_feature=pd.merge(shop_feature,cate_product,on='shop_id',how='left')
shop_feature=pd.merge(shop_feature,pid_product,on='shop_id',how='left')

In [ ]:
shop_feature.to_csv('shop_feature.csv',index=False)

# 对时间的特征进行提取

In [ ]:
holiday={'2016-08':8,'2016-09':8,'2016-10':13,'2016-11':8,'2016-12':9,
       '2017-01':12,'2017-02':9,'2017-03':8,'2017-04':11,'2017-05':10,'2017-06':8,'2017-07':10}

In [ ]:
jd_holiday={'2016-08':0,'2016-09':10,'2016-10':0,'2016-11':30,'2016-12':3,
       '2017-01':9,'2017-02':0,'2017-03':26,'2017-04':21,'2017-05':24,'2017-06':11,'2017-07':21}

In [ ]:
time_feature=feature[['shop_id','dt']]

In [ ]:
time_feature5=time_feature[time_feature.dt=='2017-04-30']
time_feature6=time_feature[time_feature.dt=='2017-04-30']
time_feature7=time_feature[time_feature.dt=='2017-04-30']

In [ ]:
time_feature5['dt']='2017-05-31'
time_feature6['dt']='2017-06-30'
time_feature7['dt']='2017-07-31'

In [ ]:
time_feature=pd.concat([time_feature,time_feature5,time_feature6,time_feature7])

In [ ]:
time_feature['dt']=pd.to_datetime(time_feature.dt)

In [ ]:
time_feature.to_csv('time_feature.csv',index=False)